In [1]:
import os
import time
import json

from pybel.parser import BelParser
import networkx as nx
import dill as pickle

# Trial 1 - Just The Grammar

In [2]:
%%time
parser = BelParser(valid_namespaces={
    "A": set('abcdef'),
    "B": set('uvwxyz')
})

CPU times: user 45.8 ms, sys: 6.8 ms, total: 52.5 ms
Wall time: 54.9 ms


The parser adds many parse actions to validate the semantics of the statement, including checking that the `namespace:name` pairs match an entry in `valid_namespaces`

In [3]:
%%time
lang = parser.language

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


In [4]:
%time res = lang.parseString('p(A:a) -> p(A:b)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 46.6 s, sys: 3.64 s, total: 50.2 s
Wall time: 50.3 s
{
  "relation": "increases",
  "object": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "b"
    }
  },
  "subject": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "a"
    }
  }
}


In [5]:
%time res = lang.parseString('p(A:c) -> p(A:d)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 8.79 ms, sys: 270 µs, total: 9.06 ms
Wall time: 9.04 ms
{
  "relation": "increases",
  "object": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "d"
    }
  },
  "subject": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "c"
    }
  }
}


In [6]:
path = os.path.expanduser('~/Desktop/lang.pickle')

In [7]:
%%time
with open(path, 'wb') as f:
    pickle.dump(lang, f) # 2.57 gigabytes, wow!

CPU times: user 3.57 s, sys: 3.89 s, total: 7.45 s
Wall time: 13.3 s


In [8]:
del lang
del parser

In [9]:
print("{:.3f} mb".format(os.path.getsize(path) / 1000000))

2570.646 mb


In [10]:
%%time
with open(path, 'rb') as f:
    lang_reloaded = pickle.load(f)

CPU times: user 2.05 s, sys: 3.89 s, total: 5.94 s
Wall time: 10.1 s


In [11]:
%time res = lang_reloaded.parseString('p(A:e) -> p(A:f)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 8.02 ms, sys: 656 µs, total: 8.68 ms
Wall time: 9.56 ms
{
  "relation": "increases",
  "object": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "f"
    }
  },
  "subject": {
    "function": "Protein",
    "identifier": {
      "namespace": "A",
      "name": "e"
    }
  }
}


In [12]:
del lang_reloaded

# Trial 2 - The Whole Class

Now, will it work on the whole parser, which contains a graph?

In [13]:
%%time
parser2 = BelParser(valid_namespaces={
    "A": set('abcdef'),
    "B": set('uvwxyz')
})

CPU times: user 29.9 ms, sys: 4.23 ms, total: 34.1 ms
Wall time: 33.3 ms


In [14]:
%time res = parser2.parseString('g(B:u) -> g(B:v)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 46.9 s, sys: 3.63 s, total: 50.5 s
Wall time: 50.6 s
{
  "relation": "increases",
  "object": {
    "function": "Gene",
    "identifier": {
      "namespace": "B",
      "name": "v"
    }
  },
  "subject": {
    "function": "Gene",
    "identifier": {
      "namespace": "B",
      "name": "u"
    }
  }
}


In [15]:
%time res = parser2.parseString('g(B:x) -> g(B:y)')

print(json.dumps(res.asDict(), indent=2))

CPU times: user 3.43 ms, sys: 44 µs, total: 3.48 ms
Wall time: 3.48 ms
{
  "relation": "increases",
  "object": {
    "function": "Gene",
    "identifier": {
      "namespace": "B",
      "name": "y"
    }
  },
  "subject": {
    "function": "Gene",
    "identifier": {
      "namespace": "B",
      "name": "x"
    }
  }
}


In [16]:
parser2.graph.edges()

[(('Gene', 'B', 'u'), ('Gene', 'B', 'v')),
 (('Gene', 'B', 'x'), ('Gene', 'B', 'y'))]

In [17]:
path2 = os.path.expanduser('~/Desktop/parser.pickle')

In [18]:
%%time
with open(path2, 'wb') as f:
    pickle.dump(parser2, f) # 2.57 gigabytes, wow!

CPU times: user 3.91 s, sys: 7.65 s, total: 11.6 s
Wall time: 18.2 s


In [19]:
del parser2

In [20]:
print("{:.3f} mb".format(os.path.getsize(path2) / 1000000))

2524.790 mb


In [21]:
%%time
with open(path2, 'rb') as f:
    parser3 = pickle.load(f) 

CPU times: user 2.29 s, sys: 5.08 s, total: 7.37 s
Wall time: 12.1 s


In [22]:
%time res = parser3.parseString('g(B:v) -> g(B:y)')

CPU times: user 3.77 ms, sys: 2.31 ms, total: 6.08 ms
Wall time: 5.96 ms


In [23]:
parser3.graph.edges()

[(('Gene', 'B', 'u'), ('Gene', 'B', 'v')),
 (('Gene', 'B', 'v'), ('Gene', 'B', 'y')),
 (('Gene', 'B', 'x'), ('Gene', 'B', 'y'))]

In [ ]:
del parser3